# Learning to identify important repositories using graphical analysis on historical data

In the [graph analysis notebook](./graph_analysis.ipynb), we explored relationships between open source projects and communities by studying graphs. We explored relations such as common contributors and project activities between different GitHub repositories.

In this notebook, our aim is to leverage the graph representation techniques we explored in the previous notebook and apply algorithms such as PageRank, Betweenness Centrality to find important emerging projects.

In order to do that, we test these algorithms on historical data of well known open source project communities such as OpenShift and Kubernetes.

## Connect to Augur database

We will be fetching the data from an Augur database which stores the GitHub data for a large number of open source repositories.

In [1]:
import pandas as pd
import psycopg2
import collections
from operator import itemgetter

import sqlalchemy as salc
import json
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from ipynb.fs.defs.graph_helper_functions import (
     get_repos,
     get_issue_contributors,
     get_pr_contributors,
     get_commit_contributors,
     get_prr_contributors,
     created_melted_dfs
)

with open("../../comm_cage.json") as config_file:
    config = json.load(config_file)

In [2]:
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

## Retrieve Available Repositories for OpenShift and Kubernetes

We start with known project communities during the years [2011-2014](https://developer.ibm.com/blogs/a-brief-history-of-red-hat-openshift/)  and color the project nodes differently as we plot the graphs. We categorize repositories into 3 buckets. 

1. **Well-known**: We categorize Kubernetes and Docker repos in this bucket. We assume that we are aware of the Kubernetes and Docker communities and we wish to reveal related emerging communities which are related to Kubernetes

2. **Emerging**: We categorize OpenShift repos in this bucket. We go forward with the assumption that there is an overlap of contributors between the OpenShift and Kubernetes repositories. We can then apply the graph algorithms for the Kubernetes repos to help discover the OpenShift community.

3. **Other Communities**: In this bucket, we will include other repositories belonging to un-related project communities which either already are important communities or are also emerging in that time frame (2011-2014). We found some popular communities which emerged in the 2011-2014 time range as well as already popular repositories such as Apache Hadoop, Apache Mesos, Node, Eclipse jetty.project.

In [3]:
with open('./red_hat_repos.txt', 'r') as f:
    rh_affiliated_projects = [line.strip() for line in f]

In [4]:
# Bucketing repositories based on a category

interesting_projects = ['https://github.com/apache/kafka',
                    'https://github.com/torvalds/linux']

In [5]:
repo_set_interesting, repo_name_set_interesting = get_repos(interesting_projects, engine)
print(repo_set_interesting, repo_name_set_interesting)

[29380, 36109] ['kafka', 'linux']


In [ ]:
repo_set_rh_affiliated, repo_name_set_rh_affiliated = get_repos(rh_affiliated_projects, engine)
print(repo_set_rh_affiliated, repo_name_set_rh_affiliated)

In [ ]:
repo_set = repo_set_rh_affiliated + repo_set_interesting 
repo_name_set = repo_name_set_rh_affiliated + repo_name_set_interesting 

In [ ]:
print(repo_set, repo_name_set)

### Retrieve Issue Contributors

We will now fetch all Issue contributors for various repositories.

In [ ]:
issue_contrib = get_issue_contributors(repo_set, engine)

In [ ]:
issue_contrib['created_at_dt'] = issue_contrib['created_at'].dt.date

In [ ]:
date_range = (issue_contrib['created_at_dt'] > pd.to_datetime('2011-01-01')) & (issue_contrib['created_at_dt'] <= pd.to_datetime('2014-06-30'))
issue_contrib = issue_contrib.loc[date_range]

In [ ]:
issue_contrib.head()

### Retrieve PR Contributors

We will now fetch all the PR contributors for various repositories.

In [ ]:
pr_contrib = get_pr_contributors(repo_set, engine)

In [ ]:
pr_contrib.head()

In [ ]:
pr_contrib['cntrb_id'].value_counts()

In [ ]:
# pr_contrib['cntrb_id'] = pr_contrib['cntrb_id'].astype('float').astype('Int64')

In [ ]:
pr_contrib['created_at_dt'] = pr_contrib['pr_created_at'].dt.date

In [ ]:
date_range = (pr_contrib['created_at_dt'] > pd.to_datetime('2011-01-01')) & (pr_contrib['created_at_dt'] <= pd.to_datetime('2014-06-30'))
pr_contrib = pr_contrib.loc[date_range]

In [ ]:
pr_contrib.head()

### Retrieve Commit Contributors

We will now fetch all the Commit contributors for various repositories.

In [ ]:
commit_contrib = get_commit_contributors(repo_set, engine)

In [ ]:
commit_contrib['cmt_date_attempted_dt'] = commit_contrib['cmt_date_attempted'].dt.date

In [ ]:
date_range = (commit_contrib['cmt_date_attempted_dt'] > pd.to_datetime('2011-01-01')) & (commit_contrib['cmt_date_attempted_dt'] <= pd.to_datetime('2014-06-30'))
commit_contrib = commit_contrib.loc[date_range]

In [ ]:
commit_contrib.head()

### Retrieve PR Reviewers 

We will now fetch all the PR Reviewers for various repositories.

In [ ]:
prr_contrib = get_prr_contributors(repo_set, engine)

In [ ]:
prr_contrib.head()

## Graph Type 1: Projects and Contributors as Nodes

In this section, we plot projects and contributors on the same graph as nodes and color them differently to see the relationships between them.

### Commit Contributors

In [ ]:
df_commit = commit_contrib.groupby(['repo_name', 'cntrb_id']).size().unstack(fill_value=0)
df_commit

In the above dataframe, each row represents a repository ID and each column represents a contributor. The dataframe contains counts for the number of times a contributor has made contributions to a particular repository. In the dataframe below `df_commit`, each contribution represents a commit. A value 0 means that a particular contributor has made no commits to the repository, and a a number x means that the contributor has made x number of commits to the repository.

In [ ]:
df_commit = df_commit.reset_index()

In [ ]:
df_commit.head()

In [ ]:
df_melted_commit = df_commit.melt(
    ['repo_name'],
    var_name = 'cntrb_id',value_name='number')

In [ ]:
df_melted_commit = df_melted_commit[df_melted_commit[df_melted_commit.columns[2]] != 0]
df_melted_commit.head()

In `df_melted_commit` we transpose the contributor IDs. Each row is a combination of a unique repository and a unique contributor and the number represents the number of times the contributor has made contributors to the particular repository.

#### Create melted dataframes for all contribution type dataframes

In [ ]:
df_melted_pr_contrib = created_melted_dfs(pr_contrib)
df_melted_pr_contrib.head()

In [ ]:
df_melted_issue_contrib = created_melted_dfs(issue_contrib)
df_melted_issue_contrib.head()

In [ ]:
df_melted_prr_contrib = created_melted_dfs(prr_contrib)
df_melted_prr_contrib.head()

### Join repositories of all contribution types

Combine dataframes of Issue contributors, PR contributors, PR Reviewers, and Commit Contributors to get aggregated contributor dataframes for each repository

In [ ]:
commit_issue = pd.concat([df_melted_commit, df_melted_issue_contrib]).groupby(["repo_name", "cntrb_id"]).sum().reset_index()

In [ ]:
comm_issue_pr_df = pd.concat([df_melted_pr_contrib, commit_issue]).groupby(["repo_name", "cntrb_id"]).sum().reset_index()

In [ ]:
merged_df = pd.concat([comm_issue_pr_df, df_melted_prr_contrib]).groupby(["repo_name", "cntrb_id"]).sum().reset_index()

In [ ]:
merged_df.rename(columns = {'number':'total_contributions'}, inplace = True)

In [ ]:
merged_df.head()

### Plot Graphs

In [ ]:
G = nx.from_pandas_edgelist(merged_df, 
                            source='repo_name',
                            target='cntrb_id',
                            edge_attr='total_contributions',
                            create_using=nx.Graph())

Repo_name = merged_df['repo_name'].to_list()
contributor_id = merged_df['cntrb_id'].to_list()

nodes = G.nodes()

colors = []
for n in nodes:
    if n in repo_name_set_rh_affiliated:
        colors.append('blue')
    elif n in repo_name_set_interesting:
        colors.append('green')
    else:
        colors.append('yellow')

node_sizes = [800 if ns in repo_name_set else 10 for ns in nodes]

fig, ax = plt.subplots(figsize=(15,15))
yellow_patch = mpatches.Patch(color='yellow', label='Contributor')
blue_patch = mpatches.Patch(color='blue', label='Red Hat Affiliated Repositories')
green_patch = mpatches.Patch(color='green', label='Interesting Repositories')

font = {"color": "k", "fontsize": 15}
ax.set_title("Graph representation of open-source repositories and their contributors", font)

ax.legend(handles=[yellow_patch, blue_patch, green_patch])
nx.draw_networkx(G, with_labels=False, node_color=colors, font_size=10, node_size=node_sizes, ax=ax)

***What do we see in this plot?***

Here we see that some of the Kubernetes, Docker & Openshift repositories seem to form a cluster. Some of the other repositories like the jettyproject also seem to have a lot of contributors, but they don't have any links/edges with the Kubernetes repositories. Lets analyze this further.

### Use PageRank and Betweenness Centrality to Subset Nodes

We can now try to run the `PageRank` algorithm to compute the ranking of the nodes in the graph based on the structure of the incoming links. 

We will also look into the betweenness centrality in the graph to compute the shortest-path betweenness centrality for nodes. It measures how often a node occurs on all shortest paths between two nodes. Here we are trying to analyze which are the common repositories that occur on all paths in the graph. 

### Page Rank

PageRank ranks important nodes by analyzing the quantity and quality of the links that point to it. In our case, links that point to repositories come from contributors. A unidirected graph from contributors to repositories will not be able to assign importances to well connected contributors. Hence we will create a bidirected graph where for each connection between a contributor and a repository, there is a 2 way arrow.

Thus, we hope that PageRank ranks important repositories by analyzing the number of contributors connected to it as well as consider how well connected those contributors are within the network.

In [ ]:
bidirect_df = merged_df.append(merged_df.rename(columns={"repo_name":"cntrb_id","cntrb_id":"repo_name"}))

In [ ]:
# Creating a directed graph to run page rank
H = nx.from_pandas_edgelist(bidirect_df, 
                            source='cntrb_id',
                            target='repo_name',
                            edge_attr='total_contributions',
                            create_using=nx.DiGraph())

nodes = H.nodes()

Repo_name = merged_df['repo_name'].to_list()
contributor_id = merged_df['cntrb_id'].to_list()
node_sizes = [800 if ns in repo_name_set else 20 for ns in nodes]

colors = []
for n in nodes:
    if n in repo_name_set_rh_affiliated:
        colors.append('blue')
    elif n in repo_name_set_interesting:
        colors.append('green')
    else:
        colors.append('yellow')

fig, ax = plt.subplots(figsize=(15,15))

font = {"color": "k", "fontsize": 15}
ax.set_title("Bidirectional graph representation of open-source repositories and their contributors", font)

ax.legend(handles=[yellow_patch, blue_patch, green_patch])
nx.draw_networkx(H, with_labels=False, node_color=colors, node_size=node_sizes, font_size=9, ax=ax)

***What do we see in this plot?***

This plot is the same graph as the last one, only this one is directed, hence the edges have arrows. Like the previous plot here too docker, openshift and kubernetes repos seem to be connected and the other community repos are disconnected from them.

In [ ]:
pageranks = nx.pagerank(H, alpha=0.85, personalization=None, max_iter=100, tol=1e-06, nstart=None, weight='weight', dangling=None)

The first input parameter of the method, H, is the NetworkX graph. Undirected graphs will be converted to a directed graph with two directed edges for each undirected edge. The second parameter, alpha, is the damping parameter for PageRank and the default value is 0.85. The fourth parameter, max_iter, is the Maximum number of iterations. The seventh parameter, weight, represents the edge attribute that should be used as the edge weight. If it’s not specified, the weight of all edges will be 1.

In [ ]:
topnodes = dict(sorted(pageranks.items(), key = itemgetter(1), reverse = True)[:20])

We want to give higher priority to outgoing nodes. So we take 50 nodes with lowest pagerank (most number of connections)

In [ ]:
key = topnodes.keys()

In [ ]:
subgraph = nx.subgraph(H, key)

In [ ]:
nodes = subgraph.nodes()
colors = []
for n in nodes:
    if n in repo_name_set_rh_affiliated:
        colors.append('blue')
    elif n in repo_name_set_interesting:
        colors.append('green')
    else:
        colors.append('yellow')

node_sizes = [1000 if ns in repo_name_set else 50 for ns in nodes]

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

font = {"color": "k", "fontsize": 15}
ax.set_title("Top 20 repositories filtered by PageRank", font)

ax.legend(handles=[yellow_patch, blue_patch, green_patch])
nx.draw_networkx(subgraph, node_color=colors, node_size=node_sizes, font_size=9, ax=ax)

***What do we see in this plot?***

Here we see that some contributor nodes have been eliminated from the graph. However, all the repository nodes that we saw previously still remain. As we plot the top 20 nodes ranked by PageRank, it seems to have filtered out a lot of contributor nodes. However, the repo nodes still are top ranked.

Lets look at what are the top 3 nodes

The PageRank algorithm measures the importance of nodes by analyzing the quantity and quality of the links that point to it. So if a repository has a lot of contributors and especially if these contributors count as important nodes, they are still ranked high. 

In [ ]:
topnodes = dict(sorted(pageranks.items(), key = itemgetter(1), reverse = True)[:3])
topnodes

In [ ]:
key = topnodes.keys()
subgraph = nx.subgraph(H, key)
nodes = subgraph.nodes()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

font = {"color": "k", "fontsize": 15}
ax.set_title("Top 3 repositories based on PageRank", font)

nx.draw_networkx(subgraph, node_color=colors, font_size=9, ax=ax)

***What do we see in this plot?***

As we plot the top 3 nodes, we see that 2 docker repos, and an unrelated community repo jetty-project show up. 

This is expected as PageRank assigns ranks to repositories by both analyzing the **number** of contributors connected to it as well as how **important those contributors** are within the network.

The docker repos and jetty-project have a much larger number of contributors which seems to be influencing the ranks assigned by PageRank. It seems like PageRank is effective in showing us well-connected and prominent repositories.
**However, PageRank may not be the most effective algorithm here to show us important emerging repositories (here openshift) in connection to existing well-known repos (here docker and kubernetes).**

### Betweenness centrality

Centrality analysis provides information about the node’s importance for an information flow or connectivity of the network. Betweenness centrality measures the extent to which a node lies on paths between other nodes in the graph. Nodes with higher betweenness have more influence within a network. Thus repositories with higher centrality scores can thought to be influential in connection to other repositories in the network.

This is a good metric for us, as using this we are able to better capture relative importance of repositories. In our case since we start with examples of well-known repos, we can use this algorithm to find other repos which are important in connection to these well-known repos.

In [ ]:
centrality = nx.betweenness_centrality(G)

In [ ]:
node_sizes = [v * 10000 for v in centrality.values()]

nodes = G.nodes()

colors = []
for n in nodes:
    if n in repo_name_set_rh_affiliated:
        colors.append('blue')
    elif n in repo_name_set_interesting:
        colors.append('green')
    else:
        colors.append('yellow')

fig, ax = plt.subplots(figsize=(15,15))

ax.legend(handles=[yellow_patch, blue_patch, green_patch])


font = {"color": "k", "fontsize": 15}
ax.set_title("Nodes scaled on Betweenness Centrality scores", font)
nx.draw_networkx(G, node_color=colors, with_labels=False, node_size=node_sizes, font_size=9,  edge_color="gainsboro", ax=ax, alpha=0.8)


***What do we see in this plot?***

The size of the nodes in the plot above indicate higher centrality scores. We see that the centrality scores highly rank the docker, kubernetes and openshift repos. 

Betweenness Centrality gives us good results and is highly ranking openshift repos in comparison to other community repos as this algorithm is able to better capture relative importance of repositories. 

By starting off with well-known repos, centrality is able to help us disvover other repos which are important in connection to these well-known repos.

## Graph Type 2: Nodes as projects edges as contributors

In this section, we represent data in another graph representation where the project repositories are represented by nodes and the edges are shared contributions between those projects

In [ ]:
merged_df.head()

In [ ]:
contributorGraph = {}
for i, row in merged_df.iterrows():
    if row['cntrb_id'] not in contributorGraph:
        contributorGraph[row['cntrb_id']] = []
    if(row['total_contributions'] > 0):
        contributorGraph[row['cntrb_id']].append((row['repo_name'], row['total_contributions']))

In [ ]:
list(contributorGraph.items())[:20]

`contributorGraph` above is a dictionary where each key is a contributor, and the value is a list of repositories the contributor has contributed to and the number of contributions it has made.


Hence, if there are more than one repositories in the list that corresponds to a single contributor key, they can be thought of as **"connected"** project repositories and we will now calculate the number of **"shared connections"** between them. 

structure of `contributorGraph` =  
{  
`contributor1`: [(`repo1`, `contributions by the contributor1 in repo 1`)],  
 `contributor2`: [(`repo2`, `contributions by the contributor2 in repo 2` ), (`repo1`, `contributions by the contributor2 in repo 1`)]  
}

**"shared connections"** constitute of *commits*, *pull requests*, *issues* and *pull request reviews* that are made by the same contributor.
We will call 2 project repositories **"connected"** if they have a **"shared connection"** between them. 
This means if they have a contributor who makes a *commit*, *pull request*, *issue* or *pull request review* in both the repositories, they count as a shared contributor and the repositories are connected. 

We track the number of shared contributions between 2 repositories for creating this graph plot.

In [ ]:
commonRepoContributionsByContributor = collections.defaultdict(int)
for key in contributorGraph:
    if len(contributorGraph[key])-1 <= 0:
        continue
    for repoContributionIndex in range(len(contributorGraph[key])-1):
        commonRepoContributionsByContributor[(contributorGraph[key][repoContributionIndex][0], contributorGraph[key][repoContributionIndex+1][0])] += contributorGraph[key][repoContributionIndex][1]+contributorGraph[key][repoContributionIndex+1][1]
print(commonRepoContributionsByContributor)

`commonRepoContributionsByContributor` is a nested dictionary consisting of dictionaries of repository pairs and their common contributions. 

structure of `commonRepoContributionsByContributor` =  
{  
(`repo1, repo2`): `PRs by same authors in repo 1 and repo 2`,  
(`repo2, repo4`): `PRs by same authors in repo 2 and repo 4`,  
(`repo2, repo5`): `PRs by same authors in repo 2 and repo 5`,   
}

In [ ]:
res = []
for key in commonRepoContributionsByContributor:
    res.append(tuple(str(k) for k in list(key)) + (commonRepoContributionsByContributor[key],))

For plotting the graph below, we pick the repositories as the nodes and let the shared contributions dictate the edge weights

In [ ]:
g = nx.Graph()
g.add_weighted_edges_from(res)

In [ ]:
nodes = g.nodes()
colors = []

for n in nodes:
    if n in repo_name_set_rh_affiliated:
        colors.append('blue')
    elif n in repo_name_set_interesting:
        colors.append('green')
    else:
        colors.append('yellow')

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

font = {"color": "k", "fontsize": 15}
ax.set_title("Graph with Nodes as projects and edges as number of contributions", font)

ax.legend(handles=[blue_patch, green_patch])
nx.draw_networkx(g, node_size=180, node_color=colors, font_size=12, ax=ax)

***What do we see in this plot?***

The above graph represents project repositories and how close or far they are to each other based on their degree of connection (number of shared contributions amongst them). If 2 nodes are close to each other, the 2 projects have a high number of shared contributions and vice versa. Each node in this graph has atleast one connection. We only plot project repositories which are connected to existing known repositories.  

We see that this graph representation effectively filters out the repositories we are most interested in seeing. **The repository "closest" to kubernetes and docker repositories are 2 OpenShift repositories "installer" followed by "source-to-image" and "osin" and the other unrelated community repositories do not appear on the plot as they are not "connected" to kubernetes**

Even if there were common contributors between the other repos and our well-known repos, the edge lengths and the distance between the nodes can be used to filter out the most connected repos that we are interested in.

Thus, this graph representation turns out to be an effective way to filter out emerging repositories in relation to already prominent communities.

## Conclusion



In this notebook, we ran an experiment on known open source GitHub repositories. We looked at a timeframe in history where a new open source community was emerging/becoming popular and we tried to track the rise of the community using graph alrgorithms.

As an example, we wanted to track the emergence of OpenShift as the downstream of Kubernetes. So we started with some Kubernetes repos, Docker repos, OpenShift repos and some other unrelated repos with the hope that as a result of the analysis, OpenShift will emerge as the community closest in connection to Kubernetes and Docker, and the other repos although popular or important in their own context do not emerge.

As a result of creating various types of graphical representations of the Github data and using algorithms like PageRank and Betweenness centrality, we saw that:

1. **PageRank is not effective in showing us important emerging OpenShift repositories in relation to Kubernetes and Docker.**

2. **Using Betweenness Centrality ranks, we are able to get OpenShift, Kubernetes and Docker repos in the top most highly ranked or central repos.**

3. **Using the graph representation type 2 where we represent nodes as projects edges as contributors, we are able to very effectively filter out the repositories we are most interested in seeing ie Kubernetes, Docker and OpenShift repos.**

    **We see that the repositories "closest" to Kubernetes and Docker repos are 3 OpenShift repositories "installer", followed by "source-to-image" followed by "osin" and the other unrelated community repositories do not appear on the plot.**